In [85]:
import numpy as np
import pandas as pd
import sklearn
import sklearn.metrics
from numpy.testing import assert_array_equal, assert_array_almost_equal, assert_equal, assert_almost_equal
from pandas.testing import assert_frame_equal
from typing import Tuple

import warnings
warnings.filterwarnings('ignore')

# Основы метрик классификации

На вход подаются 2 массива: 

* $y_{real}$ - реальные значения бинарных классов
* $y_{pred}$ - предсказанные значения бинарных классов. 

Вам необходимо посчитать, **не используя** стандартные функции, метрики: 

* $accuracy$
* $precision$
* $recall$
* $F_1$

Возвращать числа нужно именно в данном порядке.

### Sample 1
#### Input:
```python
y_real = np.array([0, 1, 0, 0, 1, 1, 1, 1])
y_pred = np.array([0, 1, 1, 0, 1, 1, 0, 0])
```
#### Output:
```python
0.625, 0.75, 0.6, 0.66
```

In [ ]:
import numpy as np

def main_metrics(y_real: np.array, y_pred: np.array) -> (float, float, float, float):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    n = len(y_real)
    for i in range(n):
        if y_real[i] == 1 and y_pred[i] == 1:
            tp+=1
        if y_real[i] == 0 and y_pred[i] == 1:
            fp+=1
        if y_real[i] == 0 and y_pred[i] == 0:
            tn+=1
        if y_real[i] == 1 and y_pred[i] == 0:
            fn+=1
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    prec = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1 = 2*(prec*recall)/(prec+recall)
    return accuracy, prec, recall, f1

In [ ]:
######################################################
y_real = np.array([0, 1, 0, 0, 1, 1, 1, 1])
y_pred = np.array([0, 1, 1, 0, 1, 1, 0, 0])

acc, pre, rec, f1 = main_metrics(y_real, y_pred)

assert np.abs(acc - sklearn.metrics.accuracy_score(y_real, y_pred)) < 0.001
assert np.abs(pre - sklearn.metrics.precision_score(y_real, y_pred)) < 0.001
assert np.abs(rec - sklearn.metrics.recall_score(y_real, y_pred)) < 0.001
assert np.abs(f1  - sklearn.metrics.f1_score(y_real, y_pred)) < 0.001
######################################################


# Основы метрик регрессии

Решаем задачу регрессии. На вход подаются 2 массива: $y_{real}$ - реальные значения функции и $y_{pred}$ - предсказанные значения функции. 

Вам необходимо посчитать, **не используя** стандартные функции, метрики: 

* $R^2score$
* $MAE$ - `mean_absolute_error`
* $MSE$ - `mean_squared_error`
* $MSLE$ - `mean_squared_log_error`

Возвращать числа нужно именно в данном порядке.

Можете сверяться с реальными метриками в `sklearn.metrics`.

Все числа в тестах больше 0, поэтому $MSLE$ будет считаться корректно.
### Sample 1
#### Input:
```python
y_real = np.array([1, 2, 3, 4, 6])
y_pred = np.array([1, 3, 2, 4, 5])
```
#### Output:
```python
0.797297, 0.6, 0.6, 0.037856
```

In [14]:
import math
import numpy as np
def reg_metrics(y_real: np.array, y_pred: np.array) -> (float, float, float, float):
    mae = 0
    mse = 0
    msle = 0
    r = 0
    s1 = 0
    s2 = 0
    m = np.mean(y_real)
    n = len(y_pred)
    for i in range(n):
        mae += abs(y_real[i] - y_pred[i])
        mse += (y_real[i] - y_pred[i])**2
        msle += (math.log(y_real[i] +1) - math.log(y_pred[i] +1))**2
        s1 += (y_real[i] - y_pred[i])**2
        s2 += (y_real[i] - m)**2
    return 1-(s1/s2), mae/n, mse/n, msle/n
    
        

print(reg_metrics(y_real, y_pred))

(0.7972972972972974, 0.6, 0.6, 0.03785687634230184)


In [12]:
######################################################
y_real = np.array([1,2,3,4,6])
y_pred = np.array([1,3,2,4,5])

r2, mae, mse, msle = reg_metrics(y_real, y_pred)

assert np.abs(r2 - sklearn.metrics.r2_score(y_real, y_pred)) < 0.001
assert np.abs(mae - sklearn.metrics.mean_absolute_error(y_real, y_pred)) < 0.001
assert np.abs(mse - sklearn.metrics.mean_squared_error(y_real, y_pred)) < 0.001
assert np.abs(msle  - sklearn.metrics.mean_squared_log_error(y_real, y_pred)) < 0.001
######################################################


NameError: name 'sklearn' is not defined

# Нахождение Roc-curve

Вам на вход даны $y_{real}$ и массив вероятностей $y_{prob} = P(y_{pred}=1)$ необходимо реализовать функцию `roc-curve`, которая вернет 2 массива различных значений $fpr$ и $tpr$, для дальнейшего построения $Roc$ кривой.

Можно считать, что все вероятности ограничены $decimal=2$ (у каждого числа не более 2-х знаков после запятой).

### Sample
#### Input:
```python
y_real = np.array([  1,   1,   0,   0,   0,   1,   0,   1,   0])
y_prob = np.array([0.8, 0.8, 0.2, 0.2, 0.6, 0.4, 0.6, 0.6, 0.4])
```
#### Output:
```python
array([0.,  0.,  0.4, 0.6, 1. ]), #fpr
array([0., 0.5, 0.75,  1., 1. ])  #tpr
```

### Sample 2
#### Input:
```python
y_real = np.array([  1,   1,   0,   0,   1,   0,   1,   0])
y_prob = np.array([0.8, 0.8, 0.2, 0.2, 0.4, 0.4, 0.6, 0.6])
```
#### Output:
```python
array([0.,  0., 0.25, 0.5, 1. ]), #fpr
array([0., 0.5, 0.75,  1., 1. ])  #tpr

или 

array([0.,  0., 0.5, 1. ]), #fpr
array([0., 0.5,  1., 1. ])  #tpr
```

Обратите внимание на 2 пример: roc кривая, которая задается ими - одинаковая. Точка, которая уходит, находится на прямой между двумя соседними, в целом такие точки можно убирать, но будут приниматься оба варианта. Функция `sklearn.metrics.roc_curve` возвращает второй вариант.

In [121]:
import numpy as np
import sklearn
def roc(y_real: np.array, y_prob: np.array):
    t = 0
    m = 1
    FPR = []
    TPR = []
    n = len(y_real)
    while t<=1:
        tn = 0
        tp = 0
        fp = 0
        fn = 0
        for i in range (n):
            el = y_real[i] 
            if el == 1 and y_prob[i] >= t:
                tp+=1
            if el == 0 and y_prob[i] >= t:
                fp+=1
            if el == 0 and y_prob[i] < t:
                tn+=1
            if el == 1 and y_prob[i] < t:
                fn+=1
        TPR.append(tp/(tp+fn))
        FPR.append(fp/(tn+fp))
        t+=0.1
      
    return np.array(FPR), np.array(TPR)
y_real = np.array([  1,   1,   0,   0,   0,   1,   0,   1,   0])
y_prob = np.array([0.8, 0.8, 0.2, 0.2, 0.6, 0.4, 0.6, 0.6, 0.4])

fpr, tpr = roc(y_real, y_prob)
fpr_true, tpr_true, _ = sklearn.metrics.roc_curve(y_real, y_prob)

print (np.abs(auc(fpr, tpr) - auc(fpr_true, tpr_true)))

0.0


In [122]:
from sklearn.metrics import auc
######################################################
y_real = np.array([  1,   1,   0,   0,   0,   1,   0,   1,   0])
y_prob = np.array([0.8, 0.8, 0.2, 0.2, 0.6, 0.4, 0.6, 0.6, 0.4])

fpr, tpr = roc(y_real, y_prob)
fpr_true, tpr_true, _ = sklearn.metrics.roc_curve(y_real, y_prob)

assert np.abs(auc(fpr, tpr) - auc(fpr_true, tpr_true)) < 0.01
######################################################
y_real = np.array([  1,   1,   0,   0,   1,   0,   1,   0])
y_prob = np.array([0.8, 0.8, 0.2, 0.2, 0.4, 0.4, 0.6, 0.6])

fpr, tpr = roc(y_real, y_prob)
fpr_true, tpr_true, _  = sklearn.metrics.roc_curve(y_real, y_prob)

assert np.abs(auc(fpr, tpr) - auc(fpr_true, tpr_true)) < 0.01
######################################################


# GridSearch

C помощью [GridSearch](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) найдите лучшие коэффициенты гиперпараметров `max_depth` и `min_samples_leaf` для классификатора [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) и верните обученный grid_search. 

* Пределы `max_depth` $(1, 10)$ 
* Пределы `min_samples_leaf` $(1, 10)$  
* Входные данные в `data/sonar.csv`
* scoring - `precision`
* cv - $5$
* Другие параметры в `DecisionTreeClassifier` не указывать.

Не нужно Shuffl-ить данные, это может повлиять на ответ и в итоге задача не зачтется.

In [123]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

def fit_gs(X: np.ndarray, y: np.ndarray) ->  GridSearchCV:
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    tree = DecisionTreeClassifier()
    params = {
        'max_depth': [1, 10],
        'min_samples_leaf': [1, 10] 
    }
    scoring=precision
    clf = GridSearchCV(tree, params, scoring='precision', cv=5)
    clf.fit

In [ ]:
# В этом задании нет открытых тестов ;)

######################################################


# Удаление Nan

Серия задач в данном модуле объединена в одну [большую задачу по предсказанию данных](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview). В ходе выполнения модуля мы будем разбирать определенные техники, которые нужны для ее решения. Настоятельно рекомендуем выполнить все шаги **по порядку**, тогда в конце вы получите решение большой реальной задачи по МЛ.

Нам даны [данные](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data) о домах выставленных на продажу. Нам необходимо решить задачу регрессии и предсказать цену продажи дома для $X_{test}$ по данным $X_{train}$ и $y_{train}$. В нашем случае  $y_{train}$ - это столбик `SalePrice`, $X_{train}$ - все остальные столбики.

На вход подается 2 считанных датафрейма **df_train**, **df_test** из файлов без изменений. 

Начальная подготовка:

* Разделить **df_train** на **X_train**(`pd.Dataframe`) и **y_train**(`pd.Series`).
* Сконкатенировать **X_train** и **df_test** в **df** по вертикали (можно ориентироваться по столбику `Id` они как раз идут по-порядку). Не забудьте обновить индекс!

Задачи:

* Заменить в **df** все Nan-ы в категориальных признаках (`object`) на строку `missing`
* Заменить в **df** все Nan-ы в числовых признаках на 0.

Вернуть из функции измененный **df**.

In [148]:
import pandas as pd

def del_nan(df_train: pd.DataFrame, df_test: pd.DataFrame) -> pd.DataFrame:
    X_train = df_train.loc[:, df_train.columns !='SalePrice']
    y_train = df_train['SalePrice']
    df = pd.concat([X_train, df_test])
    float_cols = df.select_dtypes(include=['float64']).columns
    str_cols = df.select_dtypes(include=['object']).columns

    df.loc[:, float_cols] = df.loc[:, float_cols].fillna(0)
    df.loc[:, str_cols] = df.loc[:, str_cols].fillna('missing')
    return(df)
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [86]:
######################################################
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
del_nan_df = del_nan(train, test)

assert_frame_equal(del_nan_df, pd.read_csv('data/del_nan.csv'))
######################################################


NameError: name 'del_nan' is not defined

# Порядковые категории

Вам на вход приходит **df** из предыдущей задачи.

Если внимательно изучить файл `data_description` можно понять, что многие категориальные признаки - порядковые (упорядоченное множество). Значит их можно перевести в осмысленные числа. Значит тут можно воспользоваться `LabelEncoding`.

Ваша задача: заменить в **df** категориальные признаки на числовые, для порядковых признаков.

На выходе возвращаем измененный **df**.

Чтобы слегка упростить вам жизнь, вот вам готовые словари для перевода. Однако к каким столбцам их применять - вы должны выяснить сами, изучив файл `data_description`. Каждый маппинг используется хотя бы 1 раз, а некоторые и не по одному разу.

```python
{'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'missing':0}
{'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'missing':0}
{'Gd':4, 'Av': 3, 'Mn': 2, 'No': 1, 'missing': 0}
{'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'missing': 0}
{'Typ': 8, 'Min1': 7, 'Min2': 6, 'Mod': 5, 'Maj1': 4, 'Maj2': 3, 'Sev': 2, 'Sal': 1, 'missing': 0}
{'Fin': 3, 'RFn': 2, 'Unf': 1, 'missing': 0}
{'GdPrv': 4, 'MnPrv': 3, 'GdWo': 2, 'MnWw': 1, 'missing': 0}
{'Reg': 4, 'IR1': 3, 'IR2': 2, 'IR3': 1, 'missing': 0}
{'Lvl': 4, 'Bnk': 3, 'HLS':2,'Low':1, 'missing': 0}
{'AllPub':4, 'NoSewr':3, 'NoSeWa':2, 'ELO':1, 'missing':0}
{'Gtl':3, 'Mod':2, 'Sev':1, 'missing':0}
{'SBrkr':5, 'FuseA':4, 'FuseF':3, 'FuseP':2, 'Mix':1, 'missing':0}
{'Y':3, 'P':2, 'N':1, 'missing':0}
{'Y':1, 'N':0, 'missing':0} #тут нет ошибки, все так и задумано:)
```

In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

def cat_to_num(df: pd.DataFrame) -> pd.DataFrame:
    features ={0: {'Y': 1, 'N': 0, 'missing': 0},
            1: {'Y': 3, 'P': 2, 'N': 1, 'missing': 0},
            2: {'Gtl': 3, 'Mod': 2, 'Sev': 1, 'missing': 0},
            3: {'Fin': 3, 'RFn': 2, 'Unf': 1, 'missing': 0},
            4: {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'missing': 0},
            5: {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'missing': 0},
            6: {'GdPrv': 4, 'MnPrv': 3, 'GdWo': 2, 'MnWw': 1, 'missing': 0},
            7: {'Reg': 4, 'IR1': 3, 'IR2': 2, 'IR3': 1, 'missing': 0},
            8: {'Lvl': 4, 'Bnk': 3, 'HLS': 2, 'Low': 1, 'missing': 0},
            9: {'AllPub': 4, 'NoSewr': 3, 'NoSeWa': 2, 'ELO': 1, 'missing': 0},
            10: {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'missing': 0},
            11: {'SBrkr': 5, 'FuseA': 4, 'FuseF': 3, 'FuseP': 2, 'Mix': 1,
            'missing': 0},
            12: {'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'missing': 0},
            13: {'Typ': 8, 'Min1': 7, 'Min2': 6, 'Mod': 5, 'Maj1': 4, 'Maj2': 3, 'Sev': 2, 'Sal': 1, 'missing': 0}}



    columns = {0: ['CentralAir'],
            1: ['PavedDrive'],
            2: ['LandSlope'],
            3: ['GarageFinish'],
            4: ['ExterQual', 'BsmtQual', 'KitchenQual', 'PoolQC'],
            5: ['BsmtExposure'],
            6: ['Fence'],
            7: ['LotShape'],
            8: ['LandContour'],
            9: ['Utilities'],
            10: ['ExterCond', 'HeatingQC', 'FireplaceQu', 'GarageQual', 'GarageCond', 'BsmtCond'],
            11: ['Electrical'],
            12: ['BsmtFinType1', 'BsmtFinType2'],
            13: ['Functional']}
    
    for col in columns:
        new_f = features[col]
        for c in columns[col]:
            df[c] = df[c].map(new_f)
            
    return df
            



In [9]:
######################################################
del_nan_df = pd.read_csv('data/del_nan.csv')
cat_to_num_df = cat_to_num(del_nan_df)

assert_frame_equal(cat_to_num_df, pd.read_csv('data/cat_to_num.csv'))

categorical_cols = [col for col in cat_to_num_df.columns if cat_to_num_df[col].dtypes == "object"]
assert len(categorical_cols) == 20
######################################################


NameError: name 'assert_frame_equal' is not defined

# One hot encoding

Теперь разберемся с непорядковыми категориальными признаками.

Для начала заметим признак `MSSubClass`, у которого тип `int64`, но если посмотреть в описание `data_description` можно понять, что это - категориальный признак. 

* Измените тип признака `MSSubClass` с `int64` на `object`

Теперь можно сделать `One hot encoding`:

* Найдите все колонки с категориальными признаками и составьте из них отдельный **df_oh** `pd.DataFrame` (индекс сохранить прежний)
* Применить к полученному фрейму **df_oh** функцию [`pd.get_dummies`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) (Реализует `One Hot Encoding`)
* Удалить категориальные колонки из **df** и добавить справа к **df** фрейм с `One Hot Encoding`
 
Вернуть из функции **df**

In [16]:
import pandas as pd

def one_hot(df: pd.DataFrame) -> pd.DataFrame:
    df['MSSubClass'] = df['MSSubClass'].astype(object)
    str_cols = df.select_dtypes(include=['object']).columns
    df_oh = df[str_cols]
    df.drop(columns=str_cols, axis=1, inplace=True)
    pd.get_dummies(df_oh)
    df = pd.concat([df, df_oh], ignore_index=True, axis=1)
    
    return df_oh

In [17]:
# В этой задаче нет открытых тестов ;)

######################################################
cat_to_num_df = pd.read_csv('data/cat_to_num.csv')
one_hot_df = one_hot(cat_to_num_df)
one_hot_ans = pd.read_csv('data/one_hot.csv')

print(one_hot_df)

#assert_frame_equal(one_hot_df.astype('float64').reindex(sorted(one_hot_df.columns), axis=1), 
#                    one_hot_ans.astype('float64').reindex(sorted(one_hot_ans.columns), axis=1))

#assert one_hot_df.shape[1] == 238
######################################################


     MSSubClass MSZoning Street    Alley LotConfig Neighborhood Condition1  \
0            60       RL   Pave  missing    Inside      CollgCr       Norm   
1            20       RL   Pave  missing       FR2      Veenker      Feedr   
2            60       RL   Pave  missing    Inside      CollgCr       Norm   
3            70       RL   Pave  missing    Corner      Crawfor       Norm   
4            60       RL   Pave  missing       FR2      NoRidge       Norm   
...         ...      ...    ...      ...       ...          ...        ...   
2914        160       RM   Pave  missing    Inside      MeadowV       Norm   
2915        160       RM   Pave  missing    Inside      MeadowV       Norm   
2916         20       RL   Pave  missing    Inside      Mitchel       Norm   
2917         85       RL   Pave  missing    Inside      Mitchel       Norm   
2918         60       RL   Pave  missing    Inside      Mitchel       Norm   

     Condition2 BldgType HouseStyle  ... RoofMatl Exterior1st E

C:\Users\egregious\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


# Некоррелирующие признаки 

Мы разобрались с категориальными признаками, теперь разберемся с числовыми.

Для числовых признаков можно посчитать корреляцию с правильным ответом. Если признаки слабо коррелируют, то они нам не нужны. Например колонка `Id` явно никак не влияет на стоимость дома.

Вам на вход передается изначальный **df_train** и **df** полученный из предыдущей задачи.

Ваша задача: 

* найти корреляцию всех **числовых** признаков **df_train** с признаком `SalePrice` с помощью `pd.corr`
* если абсолютное значение корреляции признака с `SalePrice` меньше $0.05$ - удалите этот признак из **df**

Верните измененный **df** и столбец корреляции признаков с признаком `SalePrice` упорядоченный по убыванию. Начало столбца корреляции выглядит следующим образом:

|               |SalePrice |
|---------------|----------|
|**SalePrice**  |1.000000  |
|**OverallQual**|0.790982  |
|**GrLivArea**  |0.708624  |
|**GarageCars** |0.640409  |
|**GarageArea** |0.623431  |
|**TotalBsmtSF**|0.613581  |
|**1stFlrSF**   |0.605852  |
|**FullBath**   |0.560664  |

Всего должно получиться 37 числовых признаков.

In [76]:
def correlation(df: pd.DataFrame, df_train: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    df_train= df_train.astype({'MSSubClass': object})
    num = df_train.select_dtypes(include=['int64', 'float64'])
    cor_df = num.corr()['SalePrice']
    for i, r in cor_df.iteritems():
        if abs(r) <  0.05:
            df.drop(i, axis=1, inplace=True)
    return cor_df.sort_values(ascending=False)


In [77]:
one_hot_df = pd.read_csv('data/one_hot.csv')
df_train = pd.read_csv('data/train.csv')

print(correlation(one_hot_df, df_train))

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePrice, dtype: float64


In [ ]:
######################################################
one_hot_df = pd.read_csv('data/one_hot.csv')
df_train = pd.read_csv('data/train.csv')

corr_df, corr = correlation(one_hot_df, df_train)

ans_corr_df = pd.read_csv('data/corr_df.csv')
ans_corr = pd.read_csv('data/corr.csv').set_index('Unnamed: 0')

assert_frame_equal(corr_df.astype('float64').reindex(sorted(corr_df.columns), axis=1),
                    ans_corr_df.astype('float64').reindex(sorted(ans_corr_df.columns), axis=1))

assert_array_almost_equal(corr.values, ans_corr.values, decimal=4)
######################################################


# Feature Engineering и Scaling

Сразу 2 простые задачки:
1.
Давайте нагенерируем несколько фич во входном фрейме **df**:

* `TotalArea` = `TotalBsmtSF` + `1stFlrSF` + `2ndFlrSF` + `GrLivArea` + `GarageArea`
* `YearAverage` = (`YearRemodAdd` + `YearBuilt`) / 2
* `LiveAreaQual` = `OverallQual` * `GrLivArea`


На выход отправьте **df** c тремя новымим столбиками. столбцы должны идти в том же порядки что указаны в списке в хвосте **df**.

2.

У стандартного и нормального масштабирования есть одна проблема: она учитывает все признаки, даже те, которые изначально некорректны (шум, выбросы). Чтобы избавитьться от шумов и выбросов и корректно масштабировать выборку необходимо использовать [RobustScaling](https://scikit-learn.org/0.18/auto_examples/preprocessing/plot_robust_scaling.html).

Ваша задача - отмасштабировать полученный фрейм с помощью `RobustScaler`. И вернуть отмасштабированный массив (да, скалирование возвращает массив, а не DataFrame).

In [92]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler

def feature_en(df: pd.DataFrame) -> pd.DataFrame:
    df['TotalArea'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF'] + df['GrLivArea'] + df['GarageArea']
    df['YearAverage'] = (df['YearRemodAdd'] + df['YearBuilt']) / 2
    df['LiveAreaQual'] = df['OverallQual'] * df['GrLivArea']
    return df



def scaling(df: pd.DataFrame) -> np.array:
    robust_scaler = RobustScaler()
    Xtr_r = robust_scaler.fit_transform(df)
    return Xtr_r

In [93]:
######################################################
corr_df = pd.read_csv('data/corr_df.csv')
feature_df = feature_en(corr_df)
ans_feature_df = pd.read_csv('data/feature_df.csv')

assert_frame_equal(feature_df.astype('float64').reindex(sorted(feature_df.columns), axis=1),
                    ans_feature_df.astype('float64').reindex(sorted(ans_feature_df.columns), axis=1))
######################################################
feature_df = pd.read_csv('data/feature_df.csv')
scale_df = scaling(feature_df)
ans_scale_df = pd.read_csv('data/scale_df.csv').values

assert_array_almost_equal(scale_df, ans_scale_df, decimal=6)
######################################################


# Смешанная модель

Отлично, теперь мы готовы обучать модель! Осталось изучить последний интересный трюк - смешанные модели.

Возьмем [2 регрессии](https://towardsdatascience.com/ridge-and-lasso-regression-a-complete-guide-with-python-scikit-learn-e20e34bcbf0b):

* Ridge
* Lasso
 
Найдем оптимальные значения $\alpha$ для обеих регрессий с помощью GridSearch.

Теперь отправим обе модели с наилучшими параметрами в класс указанный снизу. Это класс смешения моделей. В нем параметр $\beta \in [0,1]$ - это коэффициент, с которым берется ответ одного классификаторв, а ответ второго - с коэффициентом $(1 - \beta)$. Такая техника нередко позволяет добиться лучших результатов, чем одна модель.

Теперь найдем наилучшее $\beta$ для смешенной модели также с помощью GridSearch. Осталось получить **y_pred** с помощью наилучшей смешанной модели.

На вход вы получаете **X_scaled** из предыдущей задачи и **df_train** начальный. Мы подготовили за вас **X_train**, **y_train** и **X_test**. 

В задаче необходимо минимизировать метрику `neg_mean_squared_log_error`. Для удобства мы возьмем `np.log1p(y_train)` и будем минимизировать метрику `neg_mean_squared_error`. Эту метрику необходимо минимизировать у всех 3-х GridSearch.

На выход отправьте GridSearch объект смешанной модели, а также результат **y_test**. (Не забудьте его проэкспоненциировать).

Мы выдаем вам ориентировачные параметры для каждого GridSearch. Вы можете увеличить перебор, чтобы получить лучшую модель.
```python
params_ridge = {'alpha': np.arange(1, 20)}
params_lasso = {'alpha': np.logspace(-4, 3, num=8, base=10)}
params_blend = {'beta': np.linspace(0, 1, 11)}
```

Первые 2 GridSearch **не нужно** писать в функции: они могут работать достаточно долго и превысят лимит работы задачи на сервере. Найдите у себя локально наилучшие параметры и уже с этими параметрами создайте смешанную модель внутри функции. 

Также не нужно сильно увеличивать перебор для $\beta$ - того, что есть, более чем достаточно.

P.S. Осталось сохранить файл с **y_test** и отправить его в [соревнование](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/submit). 

Улучшайте свои результы пробуя другие модели и другие параметры. Уберите больше ненужных признаков, добавьте новые фичи. Экспериментируйте и дерзайте!


In [98]:
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

class BlendRegressor(BaseEstimator): # предок класса классификаторов, чтобы можно было засунуть в GridSearch
    def __init__(self, clf1, clf2, beta=0.5):
        self.clf1 = clf1 
        self.clf2 = clf2
        self.beta = beta #параметр смешивания

    def fit(self, X, y): #обучаем классификатор
        self.X_ = X
        self.y_ = y 
        self.clf1.fit(X, y)
        self.clf2.fit(X, y)
        return self

    def predict(self, X): #возвращаем значения 
        return self.clf1.predict(X) * self.beta + self.clf2.predict(X) * (1 - self.beta)

    
def learning(X_scaled: np.array, df_train: pd.DataFrame) -> (GridSearchCV, np.array):
    X_train = X_scaled[0: len(df_train),]
    X_test  = X_scaled[len(df_train): len(X_scaled)]
    y_train = np.log1p(df_train['SalePrice'])
    params_ridge = {'alpha': np.arange(1, 20)}
    params_lasso = {'alpha': np.logspace(-4, 3, num=8, base=10)}
    params_blend = {'beta': np.linspace(0, 1, 11)}
    clf = GridSearchCV(Lasso, params_lasso)
    clf.fit_transform(X_train, y_train)
    print(clf.cv_results_)
    
    
    
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    
    


In [97]:
######################################################
from sklearn.metrics import mean_squared_log_error
X_scaled = pd.read_csv('data/scale_df.csv').values
df_train = pd.read_csv('data/train.csv')
y_test_my = pd.read_csv('data/submission.csv')['SalePrice']

learning(X_scaled, df_train)



######################################################


TypeError: Cannot clone object. You should provide an instance of scikit-learn estimator instead of a class.